In [1]:
import torch 
import torch.nn as nn 

In [ ]:
'''
Attention mechanism the text generating decoder part of the network can acesss all input token selectively this mean that some input tokens are more important than others for generating 
a given output token 
This means that some input toekns are more importaant than others for genrating a given output token .
'''

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self,d_model:int,h:int,dropout:float):
        self.d_model = d_model
        self. h = h 
        self.dropout = nn.Dropout()
        self.d_k = d_model // h 
        self.w_query = nn.Linear(d_model,d_model)
        self.w_key = nn.Linear(d_model,d_model)
        self.w_value = nn.Linear(d_model,d_model)
        self.w_output = nn.Linear(d_model,d_model)
        self.dropout = nn.Dropout(dropout)
    @staticmethod
    def attention(self,q,k,v,mask):
        query = self.w_query(q) 
        key = self.w_key(k)    
        value = self.w_value(v)

        query  = query.view(query.shape[0])